# Setting up the dataframe

In [15]:
import pandas as pd
import numpy as np

# Resnet for feature extraction

In [2]:
import os

import tensorflow as tf
import keras

from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

from tqdm import tqdm
import glob

In [29]:
IM_HEIGHT = 224
IM_WIDTH = 224
NB_EPOCHS = 1
BAT_SIZE = 16
FC_SIZE = 500 # May need to train this parameter

In [33]:
labels = [x[0] for x in os.walk("./data/img/")]
labels = labels[1:]
nb_classes = len(labels)

## Subsetting data

In [62]:
fnames = [x[0] for x in os.walk("./data/small/")][1:]

def get_category(string):
    # split by underscore
    # get last element
    temp = string.split("_")
    return temp[-1]

categories = [get_category(item) for item in fnames]

In [65]:
fnames[0]

'./data/small/Above_Average_Linen_Tee'

In [70]:
from PIL import Image

im = Image.open(fnames[0])

PermissionError: [Errno 13] Permission denied: './data/small/Above_Average_Linen_Tee'

In [68]:
"./data/small/" + get_category(fnames[0])

'./data/small/Tee'

In [ ]:
for f in fnames:
    #temp = os.path.basename(f)
    temp2 = get_category(f)
    os.rename(f, )

In [63]:
set(categories)

{'Blouse',
 'Cardigan',
 'Dress',
 'Jacket',
 'Joggers',
 'Kimono',
 'Leggings',
 'Shorts',
 'Skirt',
 'Sweater',
 'Tank',
 'Tee',
 'Top'}

----

In [ ]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    for layer in base_model.layers:
        layer.trainable = False
        model.compile(optimizer='adam',    
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])

In [4]:
def prepare_training(train_dir, test_dir, imheight=IM_HEIGHT, imwidth=IM_WIDTH, batch=BAT_SIZE):
    train_datagen =  ImageDataGenerator(
    #rotation_range=30,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.2,
    #zoom_range=0.2,
    #rescale=1./255,
    horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(
        #rotation_range=30,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #shear_range=0.2,
        #zoom_range=0.2,
        #rescale=1./255,
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )
    validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=BAT_SIZE
    )
    
    return (train_generator, validation_generator)

In [ ]:
def resnet_train(train_dir, val_dir):
    
    train_gen, val_gen = prepare_training(train_dir=train_dir, test_dir=test_dir)

    base_model = ResNet50(weights='imagenet', include_top=False)
    model = add_new_last_layer(base_model, nb_classes)

    setup_to_transfer_learn(model, base_model)

    history = model.fit_generator(train_gen, epochs = NB_EPOCHS, steps_per_epoch = nb_train_samples, \
                                     validation_data=val_gen, validation_steps = nb_val_samples)
    model.save("resnet.h5")
    
    return history

h = resnet_train(train_dir, test_dir)

In [6]:
model = ResNet50(weights='imagenet')

102858752/102853048 [==============================] - ETA: 15:2 - ETA: 10:3 - ETA: 7:5 - ETA: 7: - ETA: 9: - ETA: 6: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - 

In [ ]:
def predict(test_dir, model):
    from keras.models import load_model
    test_datagen = ImageDataGenerator()
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        shuffle=False
    )
    
    model = load_model(model) # i.e. vgg16.h5
    pred = model.predict_generator(test_generator, steps=len(test_generator))
    
    return pred

preds = predict('./val_CAL', 'resnet_2048_32.h5')

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

y_true = convert_to_two(y_real)
fpr, tpr, _ = roc_curve(y_true.ravel(), preds.ravel())
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

fig2 = plt.gcf()
plt.show()
fig2.savefig("roc_32.jpg")